In [88]:
import pandas as pd

df_train = pd.read_csv('data/train.tsv', sep='\t')
df_test = pd.read_csv('data/test.tsv', sep='\t')
reviews = pd.read_csv('data/reviews.tsv', sep='\t')

In [89]:
pd.set_option('display.float_format', '{:.5f}'.format)

# Preprocessing

In [90]:
df_train = df_train[df_train['target'] > 0.9]

In [91]:
from ast import literal_eval

def parse_cords(coord_str):
    coords = literal_eval(coord_str)
    return float(coords[0]), float(coords[1])

def create_cords_features(df):
    df['longitude'] = df['coordinates'].apply(lambda x: parse_cords(x)[0])
    df['latitude'] = df['coordinates'].apply(lambda x: parse_cords(x)[1])
    df = df.drop('coordinates', axis=1)
    return df

df_test = create_cords_features(df_test)
df_train = create_cords_features(df_train)

In [92]:
df_train = df_train[
    ~(
        (df_train['latitude'] > 56.25) |   # широта выше Москвы
        (df_train['longitude'] > df_test['longitude'].max())    # долгота западнее Москвы
    )
]

In [93]:
reviews['text_len'] = reviews['text'].apply(lambda x: len(x))
reviews.head(3)

,id,text,text_len
0,43591,"Мармелад в целом неплохой, но цены завышены, м...",83
1,43591,"Не нравится, что товар выложен открыто, слишко...",99
2,43591,"Часто попадается сухой мармелад, дубовый впере...",84


In [94]:
reviews_by_id = reviews.groupby('id')['text_len'].agg(['count', 'mean', 'std']).reset_index()\
.rename(columns={'count': 'reviews_count', 'mean': 'reviews_mean', 'std': 'reviews_std'})
reviews_by_id.head(3)

,id,reviews_count,reviews_mean,reviews_std
0,1,20,132.70000,75.40215
1,2,10,172.70000,78.26174
2,3,10,179.70000,110.97652


In [95]:
base_names = (
    df_train.columns
    .str.replace(r'_300m$', '', regex=True)
    .str.replace(r'_1000m$', '', regex=True)
)
unique_base_names = base_names.drop_duplicates().tolist()

In [96]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point
from sklearn.neighbors import NearestNeighbors
import numpy as np

def add_geo_features(df):
    gdf = gpd.GeoDataFrame(df, geometry=[Point(xy) for xy in zip(df['longitude'], df['latitude'])], crs='EPSG:4326')
    gdf_m = gdf.to_crs('EPSG:32637')  # в метрах
    X_points = gdf_m.geometry.apply(lambda p: [p.x, p.y]).tolist()

    center = gpd.GeoDataFrame(geometry=[Point(37.6173, 55.7539)], crs='EPSG:4326').to_crs('EPSG:32637')
    center_point = center.geometry.iloc[0]
    df['dist_to_center'] = gdf_m.distance(center_point)

    transport = pd.concat([
        ox.features_from_place('Moscow, Russia', tags={'aeroway': 'aerodrome'}),
        ox.features_from_place('Moscow, Russia', tags={'railway': 'station'}),
        ox.features_from_place('Moscow, Russia', tags={'amenity': 'ferry_terminal'})
    ], ignore_index=True).to_crs('EPSG:32637')

    transport['geometry'] = transport.geometry.apply(
        lambda g: g.centroid if g.geom_type in ['Polygon', 'MultiPolygon'] else g
    )
    if len(transport) > 0:
        X_trans = transport.geometry.apply(lambda p: [p.x, p.y]).tolist()
        nbrs = NearestNeighbors(n_neighbors=1).fit(X_trans)
        df['dist_to_transport_hub'] = nbrs.kneighbors(X_points)[0].flatten()
    else:
        df['dist_to_transport_hub'] = np.nan

    roads = ox.graph_from_place('Moscow, Russia', network_type='drive', custom_filter='["highway"~"primary|secondary|trunk"]')
    intersections = ox.graph_to_gdfs(roads, nodes=True, edges=False).to_crs('EPSG:32637')
    major_inters = intersections[intersections['street_count'] >= 4]
    if len(major_inters) > 0:
        X_inter = major_inters.geometry.apply(lambda p: [p.x, p.y]).tolist()
        nbrs = NearestNeighbors(n_neighbors=1).fit(X_inter)
        df['dist_to_major_intersection'] = nbrs.kneighbors(X_points)[0].flatten()
    else:
        df['dist_to_major_intersection'] = np.nan

    industrial = ox.features_from_place('Moscow, Russia', tags={'landuse': 'industrial'}).to_crs('EPSG:32637')
    if len(industrial) > 0:
        industrial_union = industrial.unary_union
        df['is_industrial'] = gdf_m.geometry.apply(lambda p: industrial_union.contains(p))
    else:
        df['is_industrial'] = False

    try:
        moscow_boundary = ox.geocode_to_gdf('Moscow, Russia').to_crs('EPSG:32637').geometry.unary_union
        df['is_outside_mkad'] = ~gdf_m.geometry.apply(lambda p: moscow_boundary.contains(p))
    except:
        df['is_outside_mkad'] = df['dist_to_center'] > 15000  # fallback

    try:
        subway = ox.features_from_place('Moscow, Russia', tags={'station': 'subway'}).to_crs('EPSG:32637')
        subway = subway[~subway.geometry.isna()]  # убираем пустые геометрии
        subway = subway[subway.geometry.type == 'Point']  # только точки
    except:
        subway = gpd.GeoDataFrame(geometry=[], crs='EPSG:32637')

    if len(subway) > 0:
        X_subway = subway.geometry.apply(lambda p: [p.x, p.y]).tolist()
        nbrs_subway = NearestNeighbors(n_neighbors=1).fit(X_subway)
        df['dist_to_subway'] = nbrs_subway.kneighbors(X_points)[0].flatten()

        nbrs_count = NearestNeighbors(radius=500).fit(X_subway)
        counts = nbrs_count.radius_neighbors(X_points, return_distance=False)
        df['count_subway_500m'] = [len(c) for c in counts]
    else:
        df['dist_to_subway'] = np.nan
        df['count_subway_500m'] = 0

    return df

In [97]:
df_train_with_geo = add_geo_features(df_train)
df_test_with_geo = add_geo_features(df_test)

C:\Users\alber\AppData\Local\Temp\ipykernel_6044\513835667.py:45: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  industrial_union = industrial.unary_union
C:\Users\alber\AppData\Local\Temp\ipykernel_6044\513835667.py:51: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  moscow_boundary = ox.geocode_to_gdf('Moscow, Russia').to_crs('EPSG:32637').geometry.unary_union
C:\Users\alber\AppData\Local\Temp\ipykernel_6044\513835667.py:45: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  industrial_union = industrial.unary_union
C:\Users\alber\AppData\Local\Temp\ipykernel_6044\513835667.py:51: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  moscow_boundary = ox.geocode_to_gdf('Moscow, Russia').to_crs('EPSG:32637').geometry.unary_union


In [98]:
def add_subway_features(df):
    gdf = gpd.GeoDataFrame(df, geometry=[Point(xy) for xy in zip(df['longitude'], df['latitude'])], crs='EPSG:4326')
    gdf_m = gdf.to_crs('EPSG:32637')  # в метрах
    X_points = gdf_m.geometry.apply(lambda p: [p.x, p.y]).tolist()

    try:
        subway = ox.features_from_place('Moscow, Russia', tags={'station': 'subway'}).to_crs('EPSG:32637')
        subway = subway[~subway.geometry.isna()]  # убираем пустые геометрии
        subway = subway[subway.geometry.type == 'Point']  # только точки
    except Exception:
        subway = gpd.GeoDataFrame(geometry=[], crs='EPSG:32637')

    if len(subway) > 0:
        X_subway = subway.geometry.apply(lambda p: [p.x, p.y]).tolist()
        nbrs_subway = NearestNeighbors(n_neighbors=1).fit(X_subway)
        df['dist_to_subway'] = nbrs_subway.kneighbors(X_points)[0].flatten()

        nbrs_500 = NearestNeighbors(radius=500).fit(X_subway)
        counts_500 = nbrs_500.radius_neighbors(X_points, return_distance=False)
        df['count_subway_500m'] = [len(c) for c in counts_500]

        nbrs_1000 = NearestNeighbors(radius=1000).fit(X_subway)
        counts_1000 = nbrs_1000.radius_neighbors(X_points, return_distance=False)
        df['count_subway_1000m'] = [len(c) for c in counts_1000]
    else:
        df['dist_to_subway'] = np.nan
        df['count_subway_500m'] = 0
        df['count_subway_1000m'] = 0

    return df

In [99]:
df_train_geo = add_subway_features(df_train_with_geo)
df_test_geo = add_subway_features(df_test_with_geo)

In [100]:
import osmnx as ox
print("OSMnx version:", ox.__version__)

OSMnx version: 2.0.6


In [101]:
df_train = df_train_geo.copy()
df_test = df_test_geo.copy()

df_train = df_train.merge(reviews_by_id, on='id', how='left')
df_test = df_test.merge(reviews_by_id, on='id', how='left')

df_train.loc[:, reviews_by_id.columns.to_list()] = df_train.loc[:, reviews_by_id.columns.to_list()].fillna(value=0)
df_test.loc[:, reviews_by_id.columns.to_list()] = df_test.loc[:, reviews_by_id.columns.to_list()].fillna(value=0)

df_test.head(3)

,id,name,category,address,traffic_300m,homes_300m,works_300m,female_300m,train_ticket_order_300m,mortgage_300m,...,dist_to_transport_hub,dist_to_major_intersection,is_industrial,is_outside_mkad,dist_to_subway,count_subway_500m,count_subway_1000m,reviews_count,reviews_mean,reviews_std
0,21472,Счастье,candy_shop,"Ходынский бул., 4, Москва",62672,4709.11052,4298.12530,38987.00000,961.00000,38.00000,...,410.24409,1744.53489,False,False,410.24409,1,1,10.00000,150.00000,104.41477
1,9837,O'STIN,baby_clothes,"Носовихинское ш., 45, Реутов",110226,12987.98926,15235.25666,96081.00000,1346.00000,88.00000,...,1620.11850,3393.72031,False,True,1620.11850,0,0,10.00000,94.70000,70.59910
2,41791,Дровосек,barbershop,"Беломорская ул., 18А, корп. 2, Москва",81080,9575.24857,9463.32290,57147.00000,1506.00000,83.00000,...,160.13470,748.56641,False,False,160.13470,1,2,0.00000,0.00000,0.00000


## TF-IDF + SVD

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

grouped = reviews.groupby('id')['text'].apply(lambda x: ' '.join(x)).reset_index()
grouped.columns = ['id', 'combined_text']

vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(grouped['combined_text'])

n_components = 100
svd = TruncatedSVD(n_components=n_components, random_state=42)
X_lsa = svd.fit_transform(X_tfidf)

In [103]:
lsa_df = pd.DataFrame(X_lsa, columns=[f'topic_{i+1}' for i in range(n_components)])
lsa_df.head()

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
0,0.27640,-0.04239,0.16839,0.00086,-0.04856,-0.19697,0.05968,0.11204,0.11691,-0.08721,...,0.00389,0.00117,-0.00620,-0.00106,-0.00707,-0.01460,0.03193,0.00246,0.01948,-0.01363
1,0.22900,-0.04959,-0.04423,-0.08137,-0.06309,0.05374,-0.05059,-0.05519,-0.00918,0.00709,...,0.01174,-0.03462,0.02274,0.03869,0.00526,-0.02217,-0.01364,-0.00295,-0.02868,-0.03318
2,0.19791,0.04981,-0.01125,-0.06053,0.05184,0.01373,0.02270,0.05743,-0.00690,-0.01861,...,0.02743,0.00475,-0.05894,0.01766,0.01773,0.02396,0.05848,0.00001,0.00607,0.01585
3,0.07965,-0.01836,-0.00399,-0.00792,-0.00887,0.00438,0.01345,-0.00034,-0.01510,-0.01397,...,-0.01545,-0.00434,-0.01183,0.00816,0.00668,0.00472,-0.00030,-0.02147,-0.01127,0.01630
4,0.10056,-0.01850,-0.01293,-0.02666,-0.01516,0.01964,0.01642,0.01411,0.02648,-0.02108,...,-0.00773,0.05709,-0.02073,-0.01374,-0.01819,0.00430,0.02837,-0.02793,0.00119,-0.02870


In [104]:
result_df = pd.concat([grouped[['id']], lsa_df], axis=1)
result_df.head()

,id,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
0,1,0.27640,-0.04239,0.16839,0.00086,-0.04856,-0.19697,0.05968,0.11204,0.11691,...,0.00389,0.00117,-0.00620,-0.00106,-0.00707,-0.01460,0.03193,0.00246,0.01948,-0.01363
1,2,0.22900,-0.04959,-0.04423,-0.08137,-0.06309,0.05374,-0.05059,-0.05519,-0.00918,...,0.01174,-0.03462,0.02274,0.03869,0.00526,-0.02217,-0.01364,-0.00295,-0.02868,-0.03318
2,3,0.19791,0.04981,-0.01125,-0.06053,0.05184,0.01373,0.02270,0.05743,-0.00690,...,0.02743,0.00475,-0.05894,0.01766,0.01773,0.02396,0.05848,0.00001,0.00607,0.01585
3,4,0.07965,-0.01836,-0.00399,-0.00792,-0.00887,0.00438,0.01345,-0.00034,-0.01510,...,-0.01545,-0.00434,-0.01183,0.00816,0.00668,0.00472,-0.00030,-0.02147,-0.01127,0.01630
4,6,0.10056,-0.01850,-0.01293,-0.02666,-0.01516,0.01964,0.01642,0.01411,0.02648,...,-0.00773,0.05709,-0.02073,-0.01374,-0.01819,0.00430,0.02837,-0.02793,0.00119,-0.02870


In [105]:
df_train = df_train.merge(result_df, on='id', how='left')
df_test = df_test.merge(result_df, on='id', how='left')

df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

df_train.head()

,id,name,category,address,target,traffic_300m,homes_300m,works_300m,female_300m,train_ticket_order_300m,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
0,1365,"Городская поликлиника № 109, филиал № 2",health,"Грайвороновская ул., 18, корп. 1, Москва",4.10000,75429,16113.58247,15756.24644,51316.00000,734.00000,...,-0.02682,0.00059,-0.01053,-0.02581,-0.01712,-0.00446,0.01777,-0.00136,-0.01386,-0.02538
1,8230,Wellness Club Nebo,swimming_pool,"Пресненская наб., 12, Москва",3.60000,246535,8578.45874,31315.67279,192547.00000,4701.00000,...,-0.01483,0.02213,0.03195,0.00969,0.01073,-0.00211,-0.01096,-0.02347,0.01733,-0.00302
2,29071,Первый МГМУ имени Сеченова,higher_education_institutions,"ул. Плющиха, 57, стр. 1, Москва",3.50000,83490,12650.49240,12490.09678,56045.00000,1204.00000,...,-0.01271,0.02311,-0.03478,-0.01119,-0.00999,0.01287,-0.00299,0.00650,0.00720,-0.01751
3,22591,your SPAce,cosmetology,"просп. Маршала Жукова, 59, Москва",4.00000,43421,6413.27922,4963.05805,28705.00000,540.00000,...,0.00726,0.02097,0.01992,-0.00284,-0.02569,-0.00788,0.02520,0.02798,0.00655,-0.01398
4,27621,СоюзЦветТорг,flower_delivery,"Преображенская площадь, 6, Москва",4.20000,155094,18638.41013,19868.19057,123492.00000,2411.00000,...,-0.02368,0.01272,0.03803,-0.00579,-0.00203,0.04122,0.05117,0.02213,-0.00947,0.03410


In [106]:
def city_convertation(df):
    df['city_1'] = df['address'].str.split(',').str[-1].str.strip()
    df['city_2'] = df['address'].str.split(',').str[0].str.strip()
    
    valid_city_patterns = {
        'single_word': r'^[А-ЯЁ][а-яё]*$',
        'd_with_word': r'^д\.\s+[А-ЯЁ][а-яё]*$',
        'poselok': r'^пос[её]лок',
        'selo': r'^село\s+[А-ЯЁ][а-яё]*(\s+[А-ЯЁ][а-яё]*)*$',  # <-- добавлено
    }
    combined_pattern = '|'.join(f"({pattern})" for pattern in valid_city_patterns.values())
    mask = df['city_2'].str.contains(combined_pattern, case=True, regex=True, na=False)
    
    df['city_1'] = df['city_1'].where(~mask, df['city_2'])

    valid_city_patterns = {
        'poselok': r'пос[её]лок',  # поддерживает "поселок" и "посёлок"
        'selo': r'село\s+',
        'd_with_word': r'^д\.\s+[А-ЯЁ][а-яё]*$',  # оставляем как есть
    }

    # Создаём булевые колонки
    for col_name, pattern in valid_city_patterns.items():
        df[col_name] = df['city_1'].str.contains(pattern, case=True, regex=True, na=False)
    
    df.drop(columns=['city_2'], inplace=True)
    return df

In [107]:
df_train = city_convertation(df_train)
df_test = city_convertation(df_test)

C:\Users\alber\AppData\Local\Temp\ipykernel_6044\4276454671.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['city_2'].str.contains(combined_pattern, case=True, regex=True, na=False)
C:\Users\alber\AppData\Local\Temp\ipykernel_6044\4276454671.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['city_2'].str.contains(combined_pattern, case=True, regex=True, na=False)


In [108]:
print(len(df_test.city_1.value_counts()))
print(len(df_train.city_1.value_counts()))

258
565


# Model Selection

In [109]:
from sklearn.model_selection import train_test_split

df_train.dropna(axis=0, subset='city_1', inplace=True)

X = df_train.copy()
X_test = df_test.copy()

X_ctb = X.drop(columns=['target', 'id', 'latitude', 'longitude'])
X_test_ctb = X_test.drop(columns=['id', 'latitude', 'longitude'])

In [110]:
valid_cities = X.city_1.value_counts().head(108).index.to_list()
valid_names = X_test.name.value_counts().head(180).index.to_list()

def one_hot_top_cities_and_names(df, city_col, top_cities, name_col, top_names):
    df = df.copy()
    
    for city in top_cities:
        df[f'city_{city}'] = (df[city_col] == city).astype(int)

    for name in top_names:
        df[f'name_{name}'] = (df[name_col] == name).astype(int)
    
    df = df.drop(columns=[city_col, name_col])    
    return df

X_ctb_encoded = one_hot_top_cities_and_names(X_ctb, 'city_1', valid_cities, 'name', valid_names)
X_ctb_test_encoded = one_hot_top_cities_and_names(X_test_ctb, 'city_1', valid_cities, 'name', valid_names)

assert list(X_ctb_encoded.columns) == list(X_ctb_test_encoded.columns), "Колонки не совпадают!"
print(f"✅ Создано {len(valid_cities) + 180} one-hot колонок.")
print(f"Форма train: {X_ctb_encoded.shape}, test: {X_ctb_test_encoded.shape}")

C:\Users\alber\AppData\Local\Temp\ipykernel_6044\1118650620.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'city_{city}'] = (df[city_col] == city).astype(int)
C:\Users\alber\AppData\Local\Temp\ipykernel_6044\1118650620.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'city_{city}'] = (df[city_col] == city).astype(int)
C:\Users\alber\AppData\Local\Temp\ipykernel_6044\1118650620.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

✅ Создано 288 one-hot колонок.
Форма train: (37084, 683), test: (9276, 683)


C:\Users\alber\AppData\Local\Temp\ipykernel_6044\1118650620.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'city_{city}'] = (df[city_col] == city).astype(int)
C:\Users\alber\AppData\Local\Temp\ipykernel_6044\1118650620.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'city_{city}'] = (df[city_col] == city).astype(int)
C:\Users\alber\AppData\Local\Temp\ipykernel_6044\1118650620.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

In [111]:
X = X.drop(columns=['target', 'id', 'latitude', 'longitude', 'address'])
X_test = X_test.drop(columns=['id', 'latitude', 'longitude', 'address'])

y = df_train['target']

In [112]:
from sklearn.preprocessing import StandardScaler

def scale_features(X_train, X_test):
    for col in X_train.select_dtypes(exclude='object').columns.to_list():
        scaler = StandardScaler()
        X_train[col] = scaler.fit_transform(X_train[col].values.reshape(-1, 1)).flatten()
        X_test[col] = scaler.transform(X_test[col].values.reshape(-1, 1)).flatten()    
    return X_train, X_test

X_scaled, X_test_scaled = scale_features(X, X_test)

In [113]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=52)

## CatBoost

In [114]:
X_scaled.head()

,name,category,traffic_300m,homes_300m,works_300m,female_300m,train_ticket_order_300m,mortgage_300m,recipes_300m,online_shops_300m,...,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100,city_1,poselok,selo,d_with_word
0,"Городская поликлиника № 109, филиал № 2",health,-0.31911,0.48871,0.25006,-0.38091,-0.52527,-0.53153,-0.37250,-0.42404,...,-0.84118,-0.22730,0.88185,-0.06534,-0.71824,-1.27557,Москва,-0.16767,-0.05931,-0.11595
1,Wellness Club Nebo,swimming_pool,1.99889,-0.56439,2.17826,1.98362,2.33581,3.62922,2.47606,2.79141,...,0.52927,-0.11072,-0.54978,-1.17735,0.85993,-0.14783,Москва,-0.16767,-0.05931,-0.11595
2,Первый МГМУ имени Сеченова,higher_education_institutions,-0.20990,0.00471,-0.15469,-0.30174,-0.18630,-0.25415,-0.31900,-0.32324,...,-0.49044,0.63357,-0.15249,0.32976,0.34732,-0.87846,Москва,-0.16767,-0.05931,-0.11595
3,your SPAce,cosmetology,-0.75273,-0.86700,-1.08748,-0.75947,-0.66519,-0.42366,-0.74175,-0.68696,...,-1.26305,-0.39716,1.25243,1.41040,0.31410,-0.70079,Москва,-0.16767,-0.05931,-0.11595
4,СоюзЦветТорг,flower_delivery,0.76013,0.84158,0.75964,0.82748,0.68421,0.39308,0.85167,0.93034,...,-0.09871,2.04177,2.54644,1.11620,-0.49604,1.72469,Москва,-0.16767,-0.05931,-0.11595


In [115]:
# from catboost import CatBoostRegressor

# model = CatBoostRegressor(
#     n_estimators=8000,
#     learning_rate=0.022,
#     max_depth=6,
#     eval_metric='MAE',
#     verbose=100,
#     task_type='GPU',
#     random_seed=42,
# )

# model.fit(X_scaled, y, cat_features=[0, 1, 393])

with test

In [116]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, shuffle=True
)

In [117]:
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor

In [118]:
model = CatBoostRegressor(
    n_estimators=4000,
    learning_rate=0.022,
    max_depth=6,
    eval_metric='MAE',
    verbose=100,
    task_type='GPU',
    early_stopping_rounds=400,
    random_seed=42,
)

model.fit(
    X_train,
    y_train,
    cat_features=[0, 1, 393],
    eval_set=(X_test, y_test),
    use_best_model=True
)

y_pred = model.predict(X_test)
test_mae = mean_absolute_error(y_test, y_pred)
print(f"Test MAE: {test_mae:.4f}")

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 0.3257514	test: 0.3266063	best: 0.3266063 (0)	total: 72.3ms	remaining: 4m 49s
100:	learn: 0.2558123	test: 0.2542312	best: 0.2542312 (100)	total: 5.99s	remaining: 3m 51s
200:	learn: 0.2454491	test: 0.2452362	best: 0.2452362 (200)	total: 11.7s	remaining: 3m 42s
300:	learn: 0.2406604	test: 0.2419197	best: 0.2419197 (300)	total: 17.7s	remaining: 3m 37s
400:	learn: 0.2374548	test: 0.2400147	best: 0.2400147 (400)	total: 23.4s	remaining: 3m 29s
500:	learn: 0.2347632	test: 0.2386927	best: 0.2386846 (499)	total: 29.3s	remaining: 3m 24s
600:	learn: 0.2324054	test: 0.2376820	best: 0.2376820 (600)	total: 35.3s	remaining: 3m 19s
700:	learn: 0.2302786	test: 0.2369822	best: 0.2369811 (699)	total: 41.1s	remaining: 3m 13s
800:	learn: 0.2281876	test: 0.2362856	best: 0.2362856 (800)	total: 47.3s	remaining: 3m 8s
900:	learn: 0.2264414	test: 0.2357359	best: 0.2357359 (900)	total: 53.2s	remaining: 3m 2s
1000:	learn: 0.2247812	test: 0.2352979	best: 0.2352979 (1000)	total: 59.1s	remaining: 2m 57s
11

In [119]:
feat_imp = model.get_feature_importance(prettified=True)
low_imp = feat_imp[feat_imp['Importances'] < 0.1]['Feature Id'].tolist()
len(low_imp)

276

In [120]:
X_reshaped = X_ctb_encoded.drop(columns=low_imp)
X_test_reshaped = X_ctb_test_encoded.drop(columns=low_imp)

In [121]:
X_reshaped.head()

,category,address,homes_300m,works_300m,preschool_300m,computer_games_300m,economics_300m,game_consoles_300m,baby_food_300m,premium_class_cars_300m,...,name_Ника,name_Бико,name_Счастливый взгляд,name_Цветочный склад,name_Мойка,name_Подружки,name_Familia,name_Перекрёсток,name_Гармония,name_Росгосстрах
0,health,"Грайвороновская ул., 18, корп. 1, Москва",16113.58247,15756.24644,929.00000,64.00000,2148.00000,37.00000,74.00000,1436.00000,...,0,0,0,0,0,0,0,0,0,0
1,swimming_pool,"Пресненская наб., 12, Москва",8578.45874,31315.67279,3005.00000,198.00000,17755.00000,387.00000,190.00000,6436.00000,...,0,0,0,0,0,0,0,0,0,0
2,higher_education_institutions,"ул. Плющиха, 57, стр. 1, Москва",12650.49240,12490.09678,922.00000,56.00000,3229.00000,78.00000,84.00000,1495.00000,...,0,0,0,0,0,0,0,0,0,0
3,cosmetology,"просп. Маршала Жукова, 59, Москва",6413.27922,4963.05805,455.00000,17.00000,1842.00000,31.00000,75.00000,987.00000,...,0,0,0,0,0,0,0,0,0,0
4,flower_delivery,"Преображенская площадь, 6, Москва",18638.41013,19868.19057,1723.00000,179.00000,6680.00000,234.00000,130.00000,2996.00000,...,0,0,0,0,0,0,0,0,0,0


# Prediction

In [122]:
y_test = model.predict(X_test_scaled.fillna('Москва'))

In [ ]:
submit = pd.DataFrame({'id': df_test['id'], 'target': y_test})
submit.to_csv('cshka-pobeda-plz-5.csv', index=False)